In [1]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np
import os
from pathlib import Path
import data
import pandas as pd
import xgboost as xgb

2022-12-04 00:33:12.593549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 00:33:12.633301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-04 00:33:12.633983: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
current_path = os.path.abspath('')
BASE_PATH = Path(current_path)
def get_absolute_file_path(path):
    return os.path.join(BASE_PATH, path)

In [3]:
train_data_file_name = get_absolute_file_path('data/train.csv')
train_data = pd.read_csv(train_data_file_name)

In [4]:
train_data = data.preprocess(train_data, create_keywords=True, clean_text=True, count_abreviations=False)

In [5]:
train_data["abrev"] = train_data["text"].apply(lambda s: data.count_abreviations_func(s))

KeyboardInterrupt: 

In [ ]:
# select useful columns
# train_data_filtered = train_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)
train_data_filtered = train_data.loc[:, ["retweets_count","favorites_count","followers_count","statuses_count","friends_count",
                                 "hashtags_count","hour","verified","url_count","text_len","rt","Macron","Zemmour","Melenchon", 'abrev']]

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(train_data_filtered.drop("retweets_count", axis=1),
                                                    train_data_filtered["retweets_count"],
                                                    random_state=42, test_size=0.2)

# Standardize the data
normal_columns = train_data_filtered.drop(["hour", "verified", "Macron", "Zemmour", "Melenchon", "url_count", "rt", "retweets_count"], axis=1).columns
mu, sigma = X_train[normal_columns].mean(axis=0), X_train[normal_columns].std(axis=0)
X_train.loc[:, normal_columns] = (X_train[normal_columns] - mu) / sigma
X_eval.loc[:, normal_columns] = (X_eval[normal_columns] - mu) / sigma

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

In [ ]:
y_pred = xgb_model.predict(X_eval)
mae = mean_absolute_error(y_eval, y_pred)
print(mae)